In [771]:
import numpy as np


## Problem Definition


m=np.random.randint(10000,25000)
n=8
#create sparse matrix
mat=np.random.randint(15,40,(m,n))
p=mat[:-1,:]
p[p%7 -1 !=0]=0

#remove void rows and columns
rs=np.sum(p,axis=1)
cs=np.sum(p,axis=0)
rows=np.where(rs!=0)[0]
rows=np.append(rows,m-1)
cols=np.where(cs!=0)[0]
mat=mat[:,cols][rows,:]
    
#create supply demand
right=mat.sum(axis=1)
down=mat.sum(axis=0)
sparse=np.copy(mat)
sparse[sparse>0]=1


# sparse=np.matrix([[0, 0, 0, 1, 0, 1, 0, 1],
#        [0, 0, 0, 1, 1, 1, 1, 1],
#        [0, 0, 1, 0, 1, 1, 0, 0],
#        [1, 0, 0, 0, 0, 0, 0, 0],
#        [1, 1, 1, 0, 0, 0, 0, 0],
#        [1, 1, 1, 1, 1, 1, 1, 1]])

# down=np.reshape(np.array([92,32,93,83,78,138,55,75]), 8)
# right=np.array([96,149,90,28,81,202])

right, down


(array([ 15,  51,  15, ...,  22,  72, 169]),
 array([77568, 79505, 77100, 76675, 79491, 79459, 77028, 80140]))

In [772]:
'''
disjoint - add current and exit
equal - update existing - no need to subtract existing subsets; no effect on existing subsets; update all supersets as well
proper subset - add current and update all supersets
proper supersets - add current with updated value using all existing disjoint subsets; Update all supersets as well, if exists
overlaps - add current and add new superset
'''

def update_constraints_list(current_A, current_B, existing_A, existing_B, supplies):
    
    new_A = existing_A.copy()
    new_B = existing_B.copy()
    
    current_A_bitwise = np.squeeze(np.array(np.packbits(current_A)))
    existing_A_bitwise = np.squeeze(np.array(np.packbits(existing_A)))
    bitwise_intersection = np.bitwise_and(current_A_bitwise, existing_A_bitwise)
    
    all_disjoint_subsets = get_all_disjoint_subsets(current_A_bitwise, existing_A_bitwise, bitwise_intersection)
    all_supersets = get_all_supersets(current_A_bitwise, existing_A_bitwise, bitwise_intersection)
    all_other_overlaps = get_all_overlaps(current_A_bitwise, existing_A_bitwise, bitwise_intersection)
    
    if (current_A_bitwise == existing_A_bitwise).any():
        ## constraint with same set already exists
        all_supersets = np.concatenate([all_supersets, np.array([current_A_bitwise])])
        for current_constraint_bitwise in all_supersets:
            new_B = modify_constraint(current_constraint_bitwise, current_B, existing_A_bitwise, new_B)
        ## exit
        
    elif all_disjoint_subsets.shape[0] > 0 or all_supersets.shape[0] > 0 or all_other_overlaps.shape[0] > 0:
        ## constraint with same set doesn't exist but subsets/supersets exist
        ## Update the supersets first
        for current_constraint_bitwise in all_supersets:
            new_B = modify_constraint(current_constraint_bitwise, current_B, existing_A_bitwise, new_B)
        
        ## Update current constraint based on disjoint subsets and add it to constraints list
        current_supply = np.array([np.sum(np.array(current_A*supplies))])
        current_demand_including_subset = current_B
        for current_subset in all_disjoint_subsets:
            old_B = np.reshape(np.array(existing_B[existing_A_bitwise == current_subset]), 1)
            old_A = np.unpackbits(np.array(current_subset, dtype=np.uint8))
            old_supply = np.sum(np.array(old_A*supplies))
            old_demand = old_supply - old_B
            current_demand_including_subset = current_demand_including_subset + old_demand
        current_B_including_subset = current_supply - current_demand_including_subset
        new_A, new_B = add_constraint(current_A, current_B_including_subset, new_A, new_B)
        
        ## For oevrlap list keep creating new supersets and add them; skip if such superset already exist
        if all_other_overlaps.shape[0] > 0:
            for current_overlap in all_other_overlaps:
                new_superset_A_bitwise = np.bitwise_or(current_A_bitwise, current_overlap)
                if np.in1d(new_superset_A_bitwise, all_supersets):
                    continue
                else:
                    print(all_other_overlaps)
                    new_superset_A = np.unpackbits(np.array(new_superset_A_bitwise, dtype=np.uint8))
                    old_A = np.unpackbits(np.array(current_overlap, dtype=np.uint8))
                    old_supply = np.sum(np.array(old_A*supplies))
                    old_B = existing_B[existing_A_bitwise == current_overlap]
                    old_demand = old_supply - old_B #get old demand
                    new_superset_demand = old_demand + current_demand_including_subset #add current_B
                    new_superset_supply = np.sum(np.array(new_superset_A*supplies))
                    new_superset_B = new_superset_supply - new_superset_demand #reduce from new_supply
                    new_A, new_B = add_constraint(new_superset_A, new_superset_B, new_A, new_B) # Not modifying dynamically
                    ## exit
    else:
        ## It is from different island
        ## add current constraint
        current_supply = np.sum(np.array(current_A*supplies))
        current_B = current_supply - current_B
        new_A, new_B = add_constraint(current_A, current_B, new_A, new_B)
        ## exit
        
    return new_A, new_B

def get_all_disjoint_subsets(current_A_bitwise, existing_A_bitwise, bitwise_intersection):
    existing_A_bitwise_subsets = existing_A_bitwise[bitwise_intersection == existing_A_bitwise]
    existing_A_bitwise_subsets = np.setdiff1d(existing_A_bitwise_subsets, current_A_bitwise)
    existing_A_bitwise_subsets = list(filter(lambda x: np.sum(np.bitwise_and(x, existing_A_bitwise_subsets) == x) == 1, existing_A_bitwise_subsets))
    return np.array(existing_A_bitwise_subsets)  

def get_all_supersets(current_A_bitwise, existing_A_bitwise, bitwise_intersection):
    existing_A_bitwise_supersets = existing_A_bitwise[bitwise_intersection == current_A_bitwise]
    existing_A_bitwise_supersets = np.setdiff1d(existing_A_bitwise_supersets, current_A_bitwise)
    return existing_A_bitwise_supersets

def get_all_overlaps(current_A_bitwise, existing_A_bitwise, bitwise_intersection):
    existing_A_bitwise_subsets = existing_A_bitwise[np.logical_and.reduce((bitwise_intersection != 0, bitwise_intersection != existing_A_bitwise, bitwise_intersection != current_A_bitwise))]
    removal_set = np.array([0])
    for element1 in existing_A_bitwise_subsets:
        for element2 in existing_A_bitwise_subsets:
            if np.bitwise_and(element1, element2) == element1 and element1 != element2: #element1 is subset of element2
                if np.bitwise_or(element1, current_A_bitwise) == np.bitwise_or(element2, current_A_bitwise): #Both have same superset
                    removal_set = np.append(removal_set, element1)
    existing_A_bitwise_subsets = np.setdiff1d(existing_A_bitwise_subsets, np.unique(removal_set))        
    return existing_A_bitwise_subsets

def add_constraint(current_A, current_B, existing_A, existing_B):
    new_A = np.concatenate([existing_A, np.matrix(current_A)])
    new_B = np.concatenate([existing_B, current_B])
    return new_A, new_B

def modify_constraint(current_A_bitwise, current_B, existing_A_bitwise, existing_B):
    new_B = existing_B
    new_B[np.squeeze(np.array(existing_A_bitwise == current_A_bitwise))] -= current_B
    return new_B


import datetime
start_time = datetime.datetime.now()

## initial constraints
column_count = sparse.shape[1]
A = np.concatenate([-1*np.identity(column_count),np.identity(column_count)])
b = np.concatenate([np.zeros(column_count), down])

#sparse = sparse[[0,3],:]

## Adding first row as first constraint
existing_A = np.matrix(sparse[0, :])
existing_B = np.array([np.sum(np.array(sparse[0, :])*down) - right[0]])

## Looping for other rows
for current_row in range(1, (sparse.shape[0] - 1)):
    current_A = np.reshape(np.array(sparse[current_row, :]), column_count)
    current_B = np.reshape(right[current_row], 1)
    existing_A, existing_B = update_constraints_list(current_A, current_B, existing_A, existing_B, down)
    print(existing_A.shape, existing_B.shape)
    
end_time = datetime.datetime.now()


(2, 8) (2,)
(3, 8) (3,)
(4, 8) (4,)
(5, 8) (5,)
[65]
(7, 8) (7,)
[ 36  37 101]
[ 36  37 101]
[ 36  37 101]
(11, 8) (11,)
(12, 8) (12,)
(12, 8) (12,)
[ 36  37  38  39 101 103]
[ 36  37  38  39 101 103]
[ 36  37  38  39 101 103]
[ 36  37  38  39 101 103]
[ 36  37  38  39 101 103]
[ 36  37  38  39 101 103]
(19, 8) (19,)
[  6  39  48  53  54  65 103 117]
[  6  39  48  53  54  65 103 117]
[  6  39  48  53  54  65 103 117]
(23, 8) (23,)
[ 23  48  51  83 117 119]
[ 23  48  51  83 117 119]
[ 23  48  51  83 117 119]
[ 23  48  51  83 117 119]
[ 23  48  51  83 117 119]
[ 23  48  51  83 117 119]
(30, 8) (30,)
(30, 8) (30,)
(31, 8) (31,)
[  6  38  54  65 103 211 215 243 247]
[  6  38  54  65 103 211 215 243 247]
[  6  38  54  65 103 211 215 243 247]
[  6  38  54  65 103 211 215 243 247]
[  6  38  54  65 103 211 215 243 247]
[  6  38  54  65 103 211 215 243 247]
[  6  38  54  65 103 211 215 243 247]
[  6  38  54  65 103 211 215 243 247]
[  6  38  54  65 103 211 215 243 247]
(41, 8) (41,)
[  6  23  3

[  6   9  20  21  22  23  24  25  27  36  37  38  39  41  52  53  54  55
  56  57  59  72  73  85  87  88  89  91 101 103 105 117 119 120 121 123
 137 149 151 153 155 164 165 166 167 169 180 181 182 183 184 185 187 201
 202 203 213 215 217 218 219 229 231 233 234 235 245 247 248 249 250 251]
[  6   9  20  21  22  23  24  25  27  36  37  38  39  41  52  53  54  55
  56  57  59  72  73  85  87  88  89  91 101 103 105 117 119 120 121 123
 137 149 151 153 155 164 165 166 167 169 180 181 182 183 184 185 187 201
 202 203 213 215 217 218 219 229 231 233 234 235 245 247 248 249 250 251]
[  6   9  20  21  22  23  24  25  27  36  37  38  39  41  52  53  54  55
  56  57  59  72  73  85  87  88  89  91 101 103 105 117 119 120 121 123
 137 149 151 153 155 164 165 166 167 169 180 181 182 183 184 185 187 201
 202 203 213 215 217 218 219 229 231 233 234 235 245 247 248 249 250 251]
[  6   9  20  21  22  23  24  25  27  36  37  38  39  41  52  53  54  55
  56  57  59  72  73  85  87  88  89  91 101 103

(212, 8) (212,)
(212, 8) (212,)
(212, 8) (212,)
[  6   9  12  13  18  19  22  23  24  25  28  29  38  39  40  41  44  45
  50  51  54  55  56  57  60  61  66  67  70  72  73  76  77  82  83  86
  87  88  89  92  93  98  99 102 103 104 105 108 109 114 115 118 119 120
 121 124 125 130 131 134 137 141 146 147 150 151 153 157 162 163 166 167
 168 169 172 173 178 179 182 183 184 185 188 189 194 195 198 201 205 210
 211 214 215 217 221 226 227 230 231 233 236 237 242 243 246 247 248 249
 252 253]
[  6   9  12  13  18  19  22  23  24  25  28  29  38  39  40  41  44  45
  50  51  54  55  56  57  60  61  66  67  70  72  73  76  77  82  83  86
  87  88  89  92  93  98  99 102 103 104 105 108 109 114 115 118 119 120
 121 124 125 130 131 134 137 141 146 147 150 151 153 157 162 163 166 167
 168 169 172 173 178 179 182 183 184 185 188 189 194 195 198 201 205 210
 211 214 215 217 221 226 227 230 231 233 236 237 242 243 246 247 248 249
 252 253]
[  6   9  12  13  18  19  22  23  24  25  28  29  38  39

(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
(238, 8) (238,)
[ 65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  97  98  99 100 101 102
 103 104 105 106 107 108 109 110 111 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 129 130 131 134 136 137 138 139 140 141 142 143
 145 146 147 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163
 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191]
[ 65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  81  82  83

(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8)

...

(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8)

(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8)

(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8)

(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8)

(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8) (257,)
(257, 8)

In [773]:
existing_A, existing_B

(matrix([[0, 0, 1, ..., 0, 0, 0],
         [0, 0, 1, ..., 1, 0, 0],
         [0, 0, 0, ..., 0, 0, 1],
         ..., 
         [0, 0, 1, ..., 0, 1, 0],
         [1, 0, 0, ..., 1, 0, 0],
         [0, 1, 0, ..., 0, 0, 0]]),
 array([ 53663, 100234,  56503, 101355,  53957, 136840, 157534, 100598,
        134532, 157454, 159498,  54965,  99136, 133866, 155150, 156797,
        159951, 157796, 138545, 136536, 157779, 156552, 155706, 155103,
        136189, 156147, 137433, 157086, 135938,  88822,  54532, 157640,
        158998, 140597,  89818, 159129,  88562, 136705,  84683,  86829,
           -29, 136673, 153977, 133411, 158145, 158080, 158395, 136586,
        157236, 133396, 160317, 137679, 136221,  84544, 136944, 135925,
         85443,  98853, 133652, 154961, 154830, 158648, 132709, 158546,
        152866, 156245, 158582, 139056, 158908, 139481, 160743, 137495,
        137948, 160663, 138120, 101986, 157618, 157757, 135241, 158977,
        154200, 157475, 156538, 136578, 156025, 157860, 155

In [774]:
np.unique(existing_A, axis = 0).shape

(255, 8)

In [775]:
end_time - start_time

datetime.timedelta(0, 416, 343342)

In [705]:
end_time

datetime.datetime(2018, 2, 10, 0, 49, 54, 467812)

In [757]:
current_A = np.array([0, 0, 1, 0, 1, 1, 1, 0])
existing_A = np.matrix([[1, 1, 1, 1, 0, 0, 0, 0],
                               [1, 1, 1, 1, 1, 0, 0, 0],
                               [1, 0, 1, 0, 1, 0, 0, 0]])

current_A_bitwise = np.squeeze(np.array(np.packbits(current_A)))
existing_A_bitwise = np.squeeze(np.array(np.packbits(existing_A)))    
bitwise_intersection = np.bitwise_and(current_A_bitwise, existing_A_bitwise)
all_overlaps = get_all_overlaps(current_A_bitwise, existing_A_bitwise, bitwise_intersection)

[240 248 168]
240 240
False
True
240 248
True
True
[  0 240]
240 168
False
False
248 240
False
True
248 248
False
True
248 168
False
False
168 240
False
False
168 248
True
False
168 168
False
True


In [747]:
np.bitwise_or(240, current_A_bitwise)

246

In [758]:
all_overlaps

array([168, 248], dtype=uint8)

NameError: name 'removal_set' is not defined